<a href="https://colab.research.google.com/github/LuisguiDm450/Data_Science_Estudos/blob/main/Estudo_de_Condi%C3%A7%C3%B5es_para_Viabiliza%C3%A7%C3%A3o_de_Proteses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Estudo de condições especificas relatadas em hospital e sua possível viabilização do uso de próteses.

Este estudo faz parte de uma tese de pós-graduação de Biomecânica e é uma parceria entre a professora de Educação Física Barbara Livia dos Santos e eu, Luis Guilherme Chipoletti Esteves, como Analista de Dados.

### Objetivo

Este projeto tem o intuito de comparar as condições em que cada pessoa tem ao precisar ser atendida em um hospital, mais especificamente as a condição em que possivelmente será necessário a utilização de prótese, e também realizar a verificação das possíveis quantidades de casos onde seja necessário que o indivíduo possa vir utilizar algum produto de apoio ou completa utilização de uma prótese para suplementar uma função de seu corpo, por ter perdido em ocorrências externas.



##Observação importante --

 o levantamento de dados presente neste estudo faz parte de um projeto que encontra-se em andamento, e que pode sofrer mudanças e atualizações. Enfatiza-se  também que serão implementados novos dados importantes para o futuro.

## Obtenção dos Dados

Os dados utilizados nessa análise foram obtidos a partir do site Kaggle.com

In [ ]:
# importando as bibliotecas.

import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hospital-charges.csv to hospital-charges.csv


In [ ]:
df = pd.read_csv('hospital-charges.csv', error_bad_lines=False)

In [ ]:
# Verificando as primeiras linhas do dataset.

df.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,$31633.27,$5658.33,$4851.44


 O dataset utilizado para este estudo em específico traz relação de condições, localidade e valores que são cobrados por cada ocorrência, dando ênfase nas cobranças, como é visto pela colunas com valores de gastos, medicamento e valores total.


Utilizando a função info() para verificar as informações do dataset, como quantidade de linhas, colunas e os tipos de variáveis.

In [ ]:
# Verificando as informações dos dados do dataset, linhas e colunas.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163065 entries, 0 to 163064
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   DRG Definition                        163065 non-null  object
 1   Provider Id                           163065 non-null  int64 
 2   Provider Name                         163065 non-null  object
 3   Provider Street Address               163065 non-null  object
 4   Provider City                         163065 non-null  object
 5   Provider State                        163065 non-null  object
 6   Provider Zip Code                     163065 non-null  int64 
 7   Hospital Referral Region Description  163065 non-null  object
 8    Total Discharges                     163065 non-null  int64 
 9    Average Covered Charges              163065 non-null  object
 10   Average Total Payments               163065 non-null  object
 11  Average Medic

Primeiro vamos verificar se todas as colunas fazer uma varredura nos dados a procura de alguns problemas e realizar possíveis limpezas.

In [ ]:
# algumas colunas não serão importantes para o estudo, portanto irei retira-las do dataset.

df.drop(['Provider Id', 'Provider Name', 'Provider Street Address', 'Provider City', 'Provider State','Provider Zip Code', 'Hospital Referral Region Description'],
  axis='columns', inplace=True)

In [ ]:
# Verificando como o dataset ficou

df.head()

,DRG Definition,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,18,$31633.27,$5658.33,$4851.44


In [ ]:
# Renomear as colunas

renomear_colunas = {
    'DRG Definition' : 'Ocorrencias',
    ' Total Discharges ' : 'Total_de_casos',
    ' Average Covered Charges ' : 'Encargos_medicos',
    ' Average Total Payments ' : 'Total_de_pagamento',
    'Average Medicare Payments' : 'Total_sobre_medicacao'
}

df = df.rename(columns=renomear_colunas)

In [ ]:
df.head()

,Ocorrencias,Total_de_casos,Encargos_medicos,Total_de_pagamento,Total_sobre_medicacao
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,18,$31633.27,$5658.33,$4851.44


In [ ]:
#Verificando se há valores nulos

def valores_nulos(dataset):
  total = dataset.isnull().sum().sort_values(ascending=False)
  percent = dataset.isnull().sum()/dataset.isnull().count().sort_values(ascending=False)
  missing_data = pd.concat([total, percent], axis=1, sort=False, keys=['total', 'percent'])
  return missing_data

In [ ]:
valores_nulos(df)

,total,percent
Total_sobre_medicacao,0,0.0
Total_de_pagamento,0,0.0
Encargos_medicos,0,0.0
Total_de_casos,0,0.0
Ocorrencias,0,0.0


Nas colunas referente aos preços existe, em todas as linhas das colunas, o caractere “$”, vamos tirar eles.

In [ ]:
# Retirando o caractere "$", ja que é o primeiro caractere que aparece em cada linha, fica mais facil tirar

df['Encargos_medicos'] = df['Encargos_medicos'].apply(lambda x: x[1:])
df['Total_de_pagamento'] = df['Total_de_pagamento'].apply(lambda x: x[1:])
df['Total_sobre_medicacao'] = df['Total_sobre_medicacao'].apply(lambda x: x[1:])

In [ ]:
# Mudando para valores numericos as mesmas colunas

df['Encargos_medicos'] = pd.to_numeric(df['Encargos_medicos'])
df['Total_de_pagamento'] = pd.to_numeric(df['Total_de_pagamento'])
df['Total_sobre_medicacao'] = pd.to_numeric(df['Total_sobre_medicacao'])

In [ ]:
df.head()

,Ocorrencias,Total_de_casos,Encargos_medicos,Total_de_pagamento,Total_sobre_medicacao
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,91,32963.07,5777.24,4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,14,15131.85,5787.57,4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,24,37560.37,5434.95,4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,25,13998.28,5417.56,4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,18,31633.27,5658.33,4851.44


A coluna mais importante para nós neste momento é a coluna “Ocorrências” onde mostra quais são as ocorrências que levaram os pacientes para o hospital, seguido das demais colunas levantam as despesas geral dentro do hospital

In [ ]:
# Veridicando as condições.


df['Ocorrencias'].unique()

array(['039 - EXTRACRANIAL PROCEDURES W/O CC/MCC',
       '057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC',
       '069 - TRANSIENT ISCHEMIA',
       '064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC',
       '065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W CC',
       '066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W/O CC/MCC',
       '074 - CRANIAL & PERIPHERAL NERVE DISORDERS W/O MCC',
       '101 - SEIZURES W/O MCC', '149 - DYSEQUILIBRIUM',
       '176 - PULMONARY EMBOLISM W/O MCC',
       '177 - RESPIRATORY INFECTIONS & INFLAMMATIONS W MCC',
       '189 - PULMONARY EDEMA & RESPIRATORY FAILURE',
       '178 - RESPIRATORY INFECTIONS & INFLAMMATIONS W CC',
       '190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC',
       '191 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W CC',
       '193 - SIMPLE PNEUMONIA & PLEURISY W MCC',
       '194 - SIMPLE PNEUMONIA & PLEURISY W CC',
       '192 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W/O CC/MCC',
       '195 - SIMPLE P

Verificamos agora a quantidade de casos por cada condição para ver se existem uma grande disparidade de ocorrências entre eles.

In [ ]:
# verificando a quantidade de dados na coluna por cada condição.

df['Ocorrencias'].value_counts()

194 - SIMPLE PNEUMONIA & PLEURISY W CC                                      3023
690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC                             2989
292 - HEART FAILURE & SHOCK W CC                                            2953
392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DISORDERS W/O MCC                2950
641 - MISC DISORDERS OF NUTRITION,METABOLISM,FLUIDS/ELECTROLYTES W/O MCC    2899
                                                                            ... 
315 - OTHER CIRCULATORY SYSTEM DIAGNOSES W CC                                859
473 - CERVICAL SPINAL FUSION W/O CC/MCC                                      846
917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC                               843
251 - PERC CARDIOVASC PROC W/O CORONARY ARTERY STENT W/O MCC                 727
885 - PSYCHOSES                                                              613
Name: Ocorrencias, Length: 100, dtype: int64

##### Existem diferenças significativas entre elas de ocorrências de casos em hospitais, é perceptível que ocorrem um caso a mais que outros. Vamos então verificar quais desses casos são os que estamos procurando.

##### Apesar das condições serem inúmeras, temos que analisá-las uma por uma pra ver quais representam uma chance real de se utilizar próteses.

##### Para conseguir analisa-las, a minha parceira neste projeto, a professora Barbara Livia da Silva, comparou todos as ocorrências para verificar quais delas podem indicar que o paciente precise utilizar uma prótese dependendo da seriedade em que a ocorrência tenha acontecido, para este estudos vamos considerar todos os casos, ao mesmo tempo deixa em evidencia que são possíveis casos de utilização de próteses, pois cada caso é diferente do outro. Depois da verificação, estes são as ocorrências especificas que se enquadram em nosso estudo:





#####'460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC'
#####'473 - CERVICAL SPINAL FUSION W/O CC/MCC'
#####'469 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W CC'
#####'470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC'
#####'480 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT W MCC'
#####'481 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT W CC'
#####'536 - FRACTURES OF HIP & PELVIS W/O MCC'
#####'482 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT W/O CC/MCC'
#####'552 - MEDICAL BACK PROBLEMS W/O MCC'
#####'491 - BACK & NECK PROC EXC SPINAL FUSION W/O CC/MCC'
#####'563 - FX, SPRN, STRN & DISL EXCEPT FEMUR, HIP, PELVIS & THIGH W/O MCC'

Mas antes, precisamos ainda fazer algumas mudanças no dataset para conseguir fazer as devidas relações entre os dados.

In [ ]:
# vamos somar a quantidade de casos com suas condições

df_por_condicoes = df.groupby('Ocorrencias').sum()

In [ ]:
#por causa deste comando, a coluna 'Condicoes' se tornou o index

df_por_condicoes.head()

,Total_de_casos,Encargos_medicos,Total_de_pagamento,Total_sobre_medicacao
Ocorrencias,,,,
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,33606,32890061.73,7510416.19,5994748.69
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,30212,29193618.04,8054238.01,6847713.56
064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC,62093,80548778.68,22004682.41,20095217.23
065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W CC,106414,66917621.80,17976540.82,15387227.55
066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W/O CC/MCC,55849,42608822.01,10319457.31,8071381.63


Agora com este dataset novo, tendo em suas colunas as condições e quantidade de ocorrências, podemos traçar os diferentes casos com os que queremos encontrar.

In [ ]:
df_por_condicoes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 039 - EXTRACRANIAL PROCEDURES W/O CC/MCC to 948 - SIGNS & SYMPTOMS W/O MCC
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Total_de_casos         100 non-null    int64  
 1   Encargos_medicos       100 non-null    float64
 2   Total_de_pagamento     100 non-null    float64
 3   Total_sobre_medicacao  100 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 3.9+ KB


Agora, vamos pegar os casos específicos que separamos anteriormente, são estes casos que entram em nosso projeto e que precisamos traçar um paralelo com o total, afim de saber a porcentagem em relação a quantidade de ocorrências.


Para isso, primeiramente, vamos criar uma lista com estes casos, e depois vamos criar mais uma coluna "condicoes_selecionadas", e, utilizando a lista, iremos nomear quem esta na lista como uma variável “True” e os demais como “False”

In [ ]:
#fazendo uma lista das condições especificas que podem precisar utilizar proteses.


lista = ['460 - SPINAL FUSION EXCEPT CERVICAL W/O MCC',
         '473 - CERVICAL SPINAL FUSION W/O CC/MCC', 
         '469 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W MCC',
         '470 - MAJOR JOINT REPLACEMENT OR REATTACHMENT OF LOWER EXTREMITY W/O MCC',
         '480 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT W MCC',
         '481 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT W CC',
         '536 - FRACTURES OF HIP & PELVIS W/O MCC',
         '482 - HIP & FEMUR PROCEDURES EXCEPT MAJOR JOINT W/O CC/MCC',
         '552 - MEDICAL BACK PROBLEMS W/O MCC',
         '491 - BACK & NECK PROC EXC SPINAL FUSION W/O CC/MCC',
         '563 - FX, SPRN, STRN & DISL EXCEPT FEMUR, HIP, PELVIS & THIGH W/O MCC'
         ]

In [ ]:
#criando uma coluna para representar quais são as condições especificas

df_por_condicoes.loc[df_por_condicoes.index.isin(lista), 'condicoes_selecionadas'] = True
df_por_condicoes.condicoes_selecionadas.fillna(False)

Ocorrencias
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC                                     False
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC                          False
064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC                   False
065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W CC                    False
066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W/O CC/MCC              False
                                                                             ...  
885 - PSYCHOSES                                                              False
897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REHABILITATION THERAPY W/O MCC    False
917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC                               False
918 - POISONING & TOXIC EFFECTS OF DRUGS W/O MCC                             False
948 - SIGNS & SYMPTOMS W/O MCC                                               False
Name: condicoes_selecionadas, Length: 100, dtype: bool

In [ ]:
df_por_condicoes.head()

,Total_de_casos,Encargos_medicos,Total_de_pagamento,Total_sobre_medicacao,condicoes_selecionadas
Ocorrencias,,,,,
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,33606,32890061.73,7510416.19,5994748.69,NaN
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,30212,29193618.04,8054238.01,6847713.56,NaN
064 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W MCC,62093,80548778.68,22004682.41,20095217.23,NaN
065 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W CC,106414,66917621.80,17976540.82,15387227.55,NaN
066 - INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION W/O CC/MCC,55849,42608822.01,10319457.31,8071381.63,NaN


Por fim, utilizando o sum() em relação a última coluna selecionada, conseguimos distinguir quem esta na lista e quem não está, gerando a soma da quantidade de cada uma por suas colunas. Observando então na coluna “Total_de_casos" e que vemos o valor total de ocorrência de cada lado.

In [ ]:
#definindo em duas linhas os valores em quantidade de potencias utilizações de proteses e não utilização

df_protese = df_por_condicoes.groupby('condicoes_selecionadas', dropna=False).sum()
df_protese

,Total_de_casos,Encargos_medicos,Total_de_pagamento,Total_sobre_medicacao
condicoes_selecionadas,,,,
True,792568,7.159261e+08,1.949422e+08,1.693369e+08
NaN,6182750,5.176257e+09,1.388007e+09,1.215817e+09


São este valores os mais importante para nós, pois conseguimos agora distinguir todo o dataset das ocorrências com possíveis utilização de próteses dos demais casos onde não se tem relação com uma possível utilização de próteses

In [ ]:
# Verificando a quantidade de casos para quem esta dentro da lista e resto

print('Quantidade de casos que não se enquadrão em uma possivel utilização de protese é:', df_protese['Total_de_casos'].values[1])
print('Quantidade de casos que se enquadrão em uma possivel utilização de protese é:',  df_protese['Total_de_casos'].values[0])


Quantidade de casos que não se enquadrão em uma possivel utilização de protese é: 6182750
Quantidade de casos que se enquadrão em uma possivel utilização de protese é: 792568


In [ ]:
print("Encargos Médicos = True: "+str(round(df_protese['Encargos_medicos'][0],2)))
print("Encargos Médicos = NaN: "+str(round(df_protese['Encargos_medicos'][1],2)))

Encargos Médicos = True: 715926057.86
Encargos Médicos = NaN: 5176257187.74


In [ ]:
print("Total do Pagamento = True: "+str(round(df_protese['Total_de_pagamento'][0],2)))
print("Total do Pagamento = NaN: "+str(round(df_protese['Total_de_pagamento'][1],2)))

Total do Pagamento = True: 194942226.89
Total do Pagamento = NaN: 1388006988.99


In [ ]:
print("Total sobre o valor do Medicamento = True: "+str(round(df_protese['Total_sobre_medicacao'][0],2)))
print("Total sobre o valor do Medicamento = NaN: "+str(round(df_protese['Total_sobre_medicacao'][1],2)))

Total sobre o valor do Medicamento = True: 169336869.79
Total sobre o valor do Medicamento = NaN: 1215817299.26


Vamos então calcular a porcentagem de casos que pode ser necessário a utilização de próteses em relação ao total.

In [ ]:
#calculando a porcentagem de potencias utilização de proteses no total de casos, para isso vamos criar uma função para ser mais rapido

def porcentagem(valor_com_protese, valor_sem_protese):
  protese = valor_com_protese
  n_protese = valor_sem_protese
  total_de_casos_no_hospital = protese + n_protese

  protese_porcentagem = (protese / total_de_casos_no_hospital) * 100
  return protese_porcentagem

In [ ]:
print("porcentagem de casos com possiveis uso de protese é: " + "%.0f%%" % porcentagem(792568,6182750))
print("Encargos medicos de casos com possiveis uso de protese é: " + "%.0f%%" % porcentagem(715926057.86,5176257187.74))
print("Média de pagamentos de casos com possiveis uso de protese é: " + "%.0f%%" % porcentagem(194942226.89,1388006988.99))
print("Média de pagamentos sobre medicamentos de casos com possiveis uso de protese é: " + "%.0f%%" % porcentagem(169336869.79,1215817299.26))

porcentagem de casos com possiveis uso de protese é: 11%
Encargos medicos de casos com possiveis uso de protese é: 12%
Média de pagamentos de casos com possiveis uso de protese é: 12%
Média de pagamentos sobre medicamentos de casos com possiveis uso de protese é: 12%


Com isto, chegamos ao valor esperado. Como vemos, os casos de ocorrência em que a pessoa talvez tenha que utilizar uma próteses chega a 11% em relação ao total de casos que ocorrem em um hospital, apesar deste valor parecer pequeno, é um número significativo na vida dessas pessoas que possivelmente precisarão precisar de um cuidado maior, pelo resto de suas vidas, não so na questão de saúde mas como depender de condições externas para ter uma vida “normal”. Diferente de ocorrências onde a pessoa sai do hospital e voltar a ter uma vida normal, a vida para essas pessoas nunca mais será a mesma, tanto em questões de saúde, como questões financeiras e sociais, pois, diferente dos demais casos, estes saem do hospital com mais preocupações e responsabilidade do que todos os outros.